## Create a Masterlist of NYC subway stations and their GPS locations

1. Created a list of NYC station names (from file with Fares) -- DONE
2. Found a list of station names with GPS coordinates (be careful not to use station ENTRANCE file as the names are harder to match up) -- MAY BE ABLE TO FIND MORE COMPLETE LIST
3. Simple algorithm for matching between 1 and 2 above. 

About 1/3rd match right away.

About 1/3rd fail, but at least some could be rescued with LEVENSTEIN distances (I think). 
'KINGSTON - THROOP AVES' failed to find : 
['KINGSTON AVE-THROOP AVE',
'KINGSTON AVE/EASTERN PKWY']

About 1/3rd match too many times. See below for example:
['CANAL ST',
  'POINT (-74.00019299927328 40.71880300107709)',
  ['CANAL ST-LAFAYETTE ST EAST',
   'CANAL STREET',
   'CANAL STREET-BROADWAY',
   'CANAL STREET-LISPENARD ST',
   'CANAL STREET-VARICK STREET']

In [25]:
import pandas as pd
import numpy as np
import datetime

def nyc_stations():
    """returns list of nyc station names"""
    
    file = "Fare_Card_History_for_Metropolitan_Transportation_Authority__MTA___Beginning_2010.csv"
    df = pd.read_csv(file)
    df["Station"] = df.Station.str.strip()
    nyc_stations = list(set(df["Station"]))
    nyc_stations = sorted(nyc_stations)
    return nyc_stations

In [159]:
#the station names from the DataFrame with Fares
nyc_st = nyc_stations()
nyc_st


['103RD ST-CENTRAL PARK WEST',
 '103RD ST-ROOSEVELT AVE',
 '103RD STREET-BROADWAY',
 '103RD STREET-LEXINGTON AVE',
 '104TH STREET',
 '104TH STREET-LIBERTY AVENUE',
 '109TH STREET-LIBERTY AVENUE',
 '110TH STREET-8TH AVENUE',
 '110TH STREET-BROADWAY',
 '110TH STREET-LEXINGTON AVE',
 '111TH STREET',
 '111TH STREET-LENOX AVENUE',
 '111TH STREET-ROOSEVELT AVE',
 '116TH STREET-8TH AVENUE',
 '116TH STREET-COLUMBIA',
 '116TH STREET-LENOX AVENUE',
 '116TH STREET-LEXINGTON AVE',
 '121ST STREET',
 '125TH STREET',
 '125TH STREET & BROADWAY',
 '125TH STREET-LENOX AVENUE',
 '125TH STREET-LEXINGTON AVE',
 '135TH STREET-LENOX AVENUE',
 '135TH STREET-ST NICHOLAS AV',
 '137TH STREET-BROADWAY',
 '138TH ST-GRAND CONCOURSE',
 '138TH STREET-3RD AVE',
 '145TH STREET',
 '145TH STREET-BROADWAY',
 '145TH STREET-LENOX AVENUE',
 '148TH STREET-LENOX AVENUE',
 '149TH ST-GRAND CONCOURSE',
 '149TH STREET & 3RD AVENUE',
 '14TH STREET & 7TH AVENUE',
 '14TH STREET-6TH AVENUE',
 '14TH STREET-UNION SQUARE',
 '155TH STREET

In [155]:
#It is not clear how these stations match to our list...goint to try converting via geopy
#file = "DOITT_SUBWAY_ENTRANCE_01_13SEPT2010.csv"

#try a more generic subway station name to GPS coordinate file
file = "DOITT_SUBWAY_STATION_01_13SEPT2010.csv"
df = pd.read_csv(file)   
df2 = df.copy()
#df2
df2['NAME'] = df['NAME'].str.upper()
df2.head()

,URL,OBJECTID,NAME,the_geom,LINE,NOTES
0,http://web.mta.info/nyct/service/,1,ASTOR PL,POINT (-73.99106999861966 40.73005400028978),4-6-6 Express,"4 nights, 6-all times, 6 Express-weekdays AM s..."
1,http://web.mta.info/nyct/service/,2,CANAL ST,POINT (-74.00019299927328 40.71880300107709),4-6-6 Express,"4 nights, 6-all times, 6 Express-weekdays AM s..."
2,http://web.mta.info/nyct/service/,3,50TH ST,POINT (-73.98384899986625 40.76172799961419),1-2,"1-all times, 2-nights"
3,http://web.mta.info/nyct/service/,4,BERGEN ST,POINT (-73.97499915116808 40.68086213682956),2-3-4,"4-nights, 3-all other times, 2-all times"
4,http://web.mta.info/nyct/service/,5,PENNSYLVANIA AVE,POINT (-73.89488591154061 40.66471445143568),3-4,"4-nights, 3-all other times"


In [142]:
no_match = []
one_match = []
many_match = []
for i, row in df2.iterrows() : 
    indices = []
    indices = [s for i, s in enumerate(nyc_st) if row['NAME'] in s]
    if len(indices) == 1 :
        one_match.append([row.NAME, row.the_geom, indices])
    elif len(indices) == 0 :
        no_match.append(row.NAME)
    elif len(indices) > 1:
        many_match.append([row.NAME, row.the_geom, indices])
        
#one_match => 132
#no_match => 168
#many_match => 173

In [144]:
len(one_match)

132

In [158]:
no_match[0:10]

['CATHEDRAL PKWY (110TH ST)',
 'KINGSTON - THROOP AVES',
 'DELANCEY ST - ESSEX ST',
 '104TH-102ND STS',
 'BEACH 105TH ST',
 'INTERVALE AVE',
 '182ND-183RD STS',
 '174TH-175TH STS',
 'METS - WILLETS POINT',
 'FLUSHING - MAIN ST']

In [160]:
many_match[0:5]


[['CANAL ST',
  'POINT (-74.00019299927328 40.71880300107709)',
  ['CANAL ST-LAFAYETTE ST EAST',
   'CANAL STREET',
   'CANAL STREET-BROADWAY',
   'CANAL STREET-LISPENARD ST',
   'CANAL STREET-VARICK STREET']],
 ['50TH ST',
  'POINT (-73.98384899986625 40.76172799961419)',
  ['50TH STREET & 7TH AVENUE',
   '50TH STREET-8TH AVENUE',
   '50TH STREET-NEW UTRECHT AVE',
   'BAY 50TH STREET-STILLWELL']],
 ['BERGEN ST',
  'POINT (-73.97499915116808 40.68086213682956)',
  ['BERGEN STREET', 'BERGEN STREET-SMITH STREET']],
 ['238TH ST',
  'POINT (-73.90087000018522 40.88466700064975)',
  ['238TH ST-WHITE PLAINS ROAD', '238TH STREET-BROADWAY']],
 ['36TH ST',
  'POINT (-73.92901818461539 40.75196004401078)',
  ['36TH STREET-4TH AVENUE',
   '36TH STREET-NORTHERN BLVD',
   'EDGEMERE-BEACH 36TH STREET']]]

In [153]:

len(many_match)

173

In [38]:
from geopy.geocoders import Nominatim
geolocator = Nominatim(user_agent="TR_test")
location = geolocator.geocode("167TH ST-GRAND CONCOURSE NYC")
location[-1]


(40.8345068, -73.9176949)

In [57]:
#this is rather slow lookup...
found = {}
not_found = []
for st in nyc_st[0:] :
    new = st + " NYC"
    location = geolocator.geocode(new)
    if location : 
        print("Located: " + new)
        found[st] = location[-1]
    else :
        print("Could not locate: " + new)
        not_found.append(new)


Located: 103RD ST-CENTRAL PARK WEST NYC
Located: 103RD ST-ROOSEVELT AVE NYC
Located: 103RD STREET-BROADWAY NYC
Could not locate: 103RD STREET-LEXINGTON AVE NYC
Located: 104TH STREET NYC
Located: 104TH STREET-LIBERTY AVENUE NYC
Could not locate: 109TH STREET-LIBERTY AVENUE NYC
Located: 110TH STREET-8TH AVENUE NYC
Could not locate: 110TH STREET-BROADWAY NYC
Could not locate: 110TH STREET-LEXINGTON AVE NYC
Located: 111TH STREET NYC
Could not locate: 111TH STREET-LENOX AVENUE NYC
Located: 111TH STREET-ROOSEVELT AVE NYC
Located: 116TH STREET-8TH AVENUE NYC
Could not locate: 116TH STREET-COLUMBIA NYC
Could not locate: 116TH STREET-LENOX AVENUE NYC
Could not locate: 116TH STREET-LEXINGTON AVE NYC
Located: 121ST STREET NYC
Located: 125TH STREET NYC
Located: 125TH STREET & BROADWAY NYC
Could not locate: 125TH STREET-LENOX AVENUE NYC
Could not locate: 125TH STREET-LEXINGTON AVE NYC
Could not locate: 135TH STREET-LENOX AVENUE NYC
Located: 135TH STREET-ST NICHOLAS AV NYC
Could not locate: 137TH ST

GeocoderServiceError: [Errno 65] No route to host

In [79]:
#Successful match => but actually just found 155th street (not the crossroad)
test1 = "155th STREET - 8th AVENUE NYC"
location = geolocator.geocode(test1)
location

Location(155th Street, Cedar Manor, Queens County, NYC, New York, 11433, USA, (40.695656, -73.798302, 0.0))

In [89]:
#Successful match => but actually just found 155th street (not the crossroad)
test1 = "155TH ST/8th Aveue NYC"
location = geolocator.geocode(test1)
location

In [74]:
#Failed match
test2 = '157TH STREET-BROADWAY NYC'
location = geolocator.geocode(test2)
location

In [75]:
#Failed => succesful match (removed the 'th' from 157th)
test2 = '157 STREET-BROADWAY NYC'
location = geolocator.geocode(test2)
location

Location(Broadway, Manhattanville, Manhattan, Manhattan Community Board 9, New York County, NYC, New York, 10031, USA, (40.8208309, -73.9547233, 0.0))